**Regras de associação**

**Algoritmo Apriori**

**Veja: https://www.youtube.com/watch?v=0v6PiOpsJi8**

In [3]:
!pip install apyori
!pip install mlxtend


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.1 MB/s eta 0:00:00ta 0:00:01


In [4]:
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [9]:
#encoding = "cp1252", encoding='utf-8', encoding='latin1'etc
base= pd.read_csv('MercadoSim.csv', sep=';',encoding='cp1252', header=None)
base

,0,1,2,3,4,5,6
0,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
1,Leite,NaN,Cerveja,Pao,Manteiga,NaN,NaN
2,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
3,Leite,Cafe,NaN,Pao,Manteiga,NaN,NaN
4,NaN,NaN,Cerveja,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,Manteiga,NaN,NaN
6,NaN,NaN,NaN,Pao,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Feijao
8,NaN,NaN,NaN,NaN,NaN,Arroz,Feijao
9,NaN,NaN,NaN,NaN,NaN,Arroz,NaN


In [6]:
base.shape

(10, 7)

**Como estaremos treinando um modelo a priori, que recebe entradas em formato de lista, precisamos transformar o dataframe em uma lista de transações.**

In [10]:
transactions = []
for i in range(len(base)):
    transaction = [str(item) for item in base.iloc[i] if pd.notnull(item)]
    transactions.append(transaction)

# Obter todos os itens possíveis
all_items = sorted({item for transaction in transactions for item in transaction})

# Criar transações estendidas com itens e suas negações
extended_transactions = []
for transaction in transactions:
    new_transaction = []
    for item in all_items:
        if item in transaction:
            new_transaction.append(item)
        else:
            new_transaction.append(f'~{item}')
    extended_transactions.append(new_transaction)


In [11]:
# Transformar as transações estendidas em DataFrame binário
te = TransactionEncoder()
te_ary = te.fit(extended_transactions).transform(extended_transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Executar Apriori normalmente
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)

# Gerar regras de associação
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Ordenar por confiança decrescente
rules = rules.sort_values(by="confidence", ascending=False)


**Vamos chamar executar o algoritmo apriori e armazenar as regras obtidas**

In [12]:
# Regras onde o antecedente inclui algum ~item
rules_with_absence = rules[rules['antecedents'].apply(lambda x: any('~' in item for item in x))]

# Visualizar algumas
for _, row in rules_with_absence.head(10).iterrows():
    lhs = ', '.join(row['antecedents'])
    rhs = ', '.join(row['consequents'])
    print(f"Se NÃO leva [{lhs.replace('~', '')}], então leva [{rhs}]")
    print(f" → suporte: {row['support']:.2f}, confiança: {row['confidence']:.2f}")
    print("-" * 50)


Se NÃO leva [Feijao, Arroz, Leite], então leva [~Cerveja, ~Cafe]
 → suporte: 0.10, confiança: 1.00
--------------------------------------------------
Se NÃO leva [Pao, Manteiga, Cerveja, Feijao], então leva [~Arroz]
 → suporte: 0.30, confiança: 1.00
--------------------------------------------------
Se NÃO leva [Pao, Manteiga, Leite], então leva [~Arroz, ~Feijao]
 → suporte: 0.20, confiança: 1.00
--------------------------------------------------
Se NÃO leva [Pao, Manteiga, Arroz, Leite], então leva [~Feijao]
 → suporte: 0.20, confiança: 1.00
--------------------------------------------------
Se NÃO leva [Pao, Manteiga, Leite, Feijao], então leva [~Arroz]
 → suporte: 0.20, confiança: 1.00
--------------------------------------------------
Se NÃO leva [Pao, Manteiga, Leite], então leva [~Cerveja, ~Arroz]
 → suporte: 0.20, confiança: 1.00
--------------------------------------------------
Se NÃO leva [Pao, Manteiga, Cerveja, Leite], então leva [~Arroz]
 → suporte: 0.20, confiança: 1.00
-

In [18]:
from IPython.display import display, Markdown

def interpretar_regra(antecedents, consequents, support, confidence):
    lhs = ', '.join([item.replace('~', 'NÃO leva ') if '~' in item else f'leva {item}' for item in antecedents])
    rhs = ', '.join([item.replace('~', 'NÃO leva ') if '~' in item else f'leva {item}' for item in consequents])
    texto = f"**Se {lhs}, então {rhs}**  \n"
    texto += f"• Suporte: `{support:.2f}` — presente em {support*100:.1f}% das transações.  \n"
    texto += f"• Confiança: `{confidence:.2f}` — quando {lhs.lower()}, isso ocorre em {confidence*100:.1f}% dos casos.  \n"
    return texto

# Filtra regras com ausência no antecedente
regras_negativas = rules[rules['antecedents'].apply(lambda x: any('~' in i for i in x))]

# Mostra as 10 regras mais confiáveis
top_regras = regras_negativas.sort_values(by='confidence', ascending=False).head(10)

for _, row in top_regras.iterrows():
    explicacao = interpretar_regra(row['antecedents'], row['consequents'], row['support'], row['confidence'])
    display(Markdown(explicacao))
    print("-" * 60)


**Se leva Feijao, NÃO leva Arroz, NÃO leva Leite, então NÃO leva Cerveja, NÃO leva Cafe**  
• Suporte: `0.10` — presente em 10.0% das transações.  
• Confiança: `1.00` — quando leva feijao, não leva arroz, não leva leite, isso ocorre em 100.0% dos casos.  


------------------------------------------------------------


**Se NÃO leva Cerveja, leva Feijao, NÃO leva Leite, NÃO leva Manteiga, então NÃO leva Pao, NÃO leva Cafe**  
• Suporte: `0.20` — presente em 20.0% das transações.  
• Confiança: `1.00` — quando não leva cerveja, leva feijao, não leva leite, não leva manteiga, isso ocorre em 100.0% dos casos.  


------------------------------------------------------------


**Se NÃO leva Feijao, leva Cerveja, NÃO leva Leite, NÃO leva Pao, NÃO leva Manteiga, então NÃO leva Cafe**  
• Suporte: `0.10` — presente em 10.0% das transações.  
• Confiança: `1.00` — quando não leva feijao, leva cerveja, não leva leite, não leva pao, não leva manteiga, isso ocorre em 100.0% dos casos.  


...

------------------------------------------------------------


**Se NÃO leva Cerveja, NÃO leva Pao, leva Feijao, NÃO leva Manteiga, então NÃO leva Leite, NÃO leva Cafe**  
• Suporte: `0.20` — presente em 20.0% das transações.  
• Confiança: `1.00` — quando não leva cerveja, não leva pao, leva feijao, não leva manteiga, isso ocorre em 100.0% dos casos.  


------------------------------------------------------------


**Se NÃO leva Pao, leva Feijao, NÃO leva Manteiga, então NÃO leva Cerveja, NÃO leva Leite, NÃO leva Cafe**  
• Suporte: `0.20` — presente em 20.0% das transações.  
• Confiança: `1.00` — quando não leva pao, leva feijao, não leva manteiga, isso ocorre em 100.0% dos casos.  


------------------------------------------------------------
